In [2]:
#作成したテンソルを読み込み
import torch
X_train = torch.load('/content/drive/MyDrive/Colab Notebooks/chapter08/X_train.pt')
X_test = torch.load('/content/drive/MyDrive/Colab Notebooks/chapter08/X_test.pt')
X_valid = torch.load('/content/drive/MyDrive/Colab Notebooks/chapter08/X_valid.pt')
Y_train = torch.load('/content/drive/MyDrive/Colab Notebooks/chapter08/Y_train.pt')
Y_test = torch.load('/content/drive/MyDrive/Colab Notebooks/chapter08/Y_test.pt')
Y_valid = torch.load('/content/drive/MyDrive/Colab Notebooks/chapter08/Y_valid.pt')

In [3]:
# モデルの構築(nnはニューラルネットワーク)
from torch import nn

class SLNet(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.fc = nn.Linear(input_size, output_size) #全結合層の作成

    def forward(self, x):
        logits = self.fc(x)                          #フォワードパスの作成
        return logits

#入力300,出力4のサイズでインスタンスを作成
model = SLNet(300, 4)
print(model)

SLNet(
  (fc): Linear(in_features=300, out_features=4, bias=True)
)


In [4]:
# データセットを作成する
import torch.utils.data as data

#データセットのクラスを作成
class NewsDataset(data.Dataset):
    """
    newsのDatasetクラス

    Attributes
    ----------------------------
    X : テンソル
        単語ベクトルの平均をまとめたテンソル
    y : テンソル
        カテゴリをラベル化したテンソル
    phase : 'train' or 'val'
        学習か訓練かを設定する
    """
    def __init__(self, X, y, phase='train'):
        self.X = X
        self.y = y
        self.phase = phase

    def __len__(self):
        """全データサイズを返す"""
        return len(self.y)

    def __getitem__(self, idx):
        """idxに対応するテンソル形式のデータとラベルを取得"""
        return self.X[idx], self.y[idx]

train_dataset = NewsDataset(X_train, Y_train, phase='train')
valid_dataset = NewsDataset(X_valid, Y_valid, phase='val')
test_dataset = NewsDataset(X_test, Y_test, phase='val')

# 動作確認
idx = 0
print(train_dataset.__getitem__(idx)[0].size())
print(train_dataset.__getitem__(idx)[1])
print(valid_dataset.__getitem__(idx)[0].size())
print(valid_dataset.__getitem__(idx)[1])
print(test_dataset.__getitem__(idx)[0].size())
print(test_dataset.__getitem__(idx)[1])

torch.Size([300])
tensor(2)
torch.Size([300])
tensor(3)
torch.Size([300])
tensor(2)


In [5]:
# DataLoaderを作成(Dataloaderはdatasetsからバッチごとに取り出すことを目的とする)
batch_size = 1

train_dataloader = data.DataLoader(                                              #ここには10671個のバッチが含まれている
            train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = data.DataLoader(
            valid_dataset, batch_size=len(valid_dataset), shuffle=False)
test_dataloader = data.DataLoader(
            test_dataset, batch_size=len(test_dataset), shuffle=False)

dataloaders_dict = {'train': train_dataloader,
                    'val': valid_dataloader,
                    'test': test_dataloader,
                   }

# 動作確認
batch_iter = iter(dataloaders_dict['train']) #バッチからデータを取り出す
inputs, labels = next(batch_iter)
print(inputs.size())
print(labels)

torch.Size([1, 300])
tensor([0])


In [7]:
import time
from tqdm import tqdm
# 学習用の関数を定義
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # 初期設定
    # GPUが使えるか確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス:", device)

    # ネットワークをgpuへ
    net.to(device)

    # ネットワークがある程度固定なら高速化させる
    torch.backends.cudnn.benchmark = True

    train_loss = []
    train_acc = []
    valid_loss = []
    valid_acc = []
    # epochのループ
    for epoch in range(num_epochs):
        # 開始時刻の記録
        start = time.time()
        print('Epoch {} / {}'.format(epoch + 1, num_epochs))
        print('--------------------------------------------')

        # epochごとの学習と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train() # 訓練モード
            else:
                net.eval() # 検証モード

            epoch_loss = 0.0 # epochの損失和
            epoch_corrects = 0 # epochの正解数

            # データローダーからミニバッチを取り出すループ
            for inputs, labels in tqdm(dataloaders_dict[phase]):
                # GPUが使えるならGPUにおっくる
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad() # optimizerを初期化

                # 順伝播計算(forward)
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels) # 損失を計算
                    _, preds = torch.max(outputs, 1) # ラベルを予想

                    # 訓練時は逆伝播
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # イテレーション結果の計算
                    # lossの合計を更新
                    epoch_loss += loss.item() * inputs.size(0)
                    # 正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率の表示
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)
            if phase == 'train':
                train_loss.append(epoch_loss)
                train_acc.append(epoch_acc)
            else:
                valid_loss.append(epoch_loss)
                valid_acc.append(epoch_acc)

            print('{} Loss: {:.4f}, Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
        # 修了時刻の記録
        end = time.time()
        calc_time = end - start
        print('batch_size {} calc_time: {:.4f} sec'.format(batch_size, calc_time))
    return train_loss, train_acc, valid_loss, valid_acc, calc_time

# 学習を実行する
batch_sizes = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048]                           #今回検証するバッチサイズを保持
cpu_times = []                                                                              #処理時間を保存するための配列
for batch_size in batch_sizes:                                                              #バッチサイズによってループ
    print('batch_size: {}'.format(batch_size))
    # DataLoaderを作成
    train_dataloader = data.DataLoader(
                train_dataset, batch_size=batch_size, shuffle=True)
    valid_dataloader = data.DataLoader(
                valid_dataset, batch_size=len(valid_dataset), shuffle=False)
    test_dataloader = data.DataLoader(
                test_dataset, batch_size=len(test_dataset), shuffle=False)

    dataloaders_dict = {'train': train_dataloader,
                        'val': valid_dataloader,
                        'test': test_dataloader,
                       }
    # モデルの定義
    net = SLNet(300, 4)
    net.train()

    # 損失関数の定義
    criterion = nn.CrossEntropyLoss()

    # 最適化手法の定義
    optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

    num_epochs = 1
    train_loss, train_acc, valid_loss, valid_acc, calc_time = \
                        train_model(net, dataloaders_dict, criterion, optimizer,
                                    num_epochs=num_epochs)
    cpu_times.append(calc_time)

batch_size: 1
使用デバイス: cuda:0
Epoch 1 / 1
--------------------------------------------


100%|██████████| 10672/10672 [00:12<00:00, 850.96it/s]


train Loss: 0.4165, Acc: 0.8604


100%|██████████| 1/1 [00:00<00:00, 17.74it/s]


val Loss: 0.3155, Acc: 0.8936
batch_size 1 calc_time: 12.6492 sec
batch_size: 2
使用デバイス: cuda:0
Epoch 1 / 1
--------------------------------------------


...

val Loss: 1.2944, Acc: 0.5802
batch_size 1024 calc_time: 0.1018 sec
batch_size: 2048
使用デバイス: cuda:0
Epoch 1 / 1
--------------------------------------------


100%|██████████| 6/6 [00:00<00:00, 29.41it/s]


train Loss: 1.3594, Acc: 0.4025


100%|██████████| 1/1 [00:00<00:00, 93.67it/s]

val Loss: 1.3338, Acc: 0.4415
batch_size 2048 calc_time: 0.2289 sec
